In [1]:
## Imports

# import python libraries  
import numpy as np
import pandas as pd
import sys
import os
import time
import pickle
import tensorflow as tf
import time
from importlib import reload
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers

from scripts.splitting import split
from scripts.data_wrangling import import_phos
from scripts.feature_selection import fs_functional_score, fs_atlas_landmark, true_phosphosite_filter
from scripts.data_preparation import data_prep
from scripts.data_selection import data_indexing
from scripts.mean_model import MeanModel
from scripts.tf_DRP_model import train_model, train_model_cv

tf.keras.utils.set_random_seed(42) # set random seeds for Python, NumPy, and TensorFlow

print('Imports: Done')



## Data Wrangling

# Import and wrangle phosphoproteomics and GDSC1 drug data
phos_df, drug_df, drug_matrix, _all_cls, _all_drugs, common_ind  = import_phos()

## Feature Selection 

# create list of landmark phosphosites
phosphosites = fs_functional_score(phos_df,95)
#phosphosites = fs_atlas_landmark(phos_df,80)

# remove false positive phosphosites
filtered_phosphosites = true_phosphosite_filter(phosphosites)

# reduce remaining features down by feature selection
phos_df = phos_df.filter(filtered_phosphosites,axis=1)

## Data Preparation & Selection
x_drug, x_all, y_series = data_prep(drug_df,phos_df,common_ind)
drug_cl_pairs =  y_series.index

# Cancer blind train-test split
rand_seed = 42 # set random seed for train-test split
train_size = 0.75 # set train-test fraction

train_pairs, test_pairs = split(rand_seed, _all_cls, _all_drugs, drug_cl_pairs, 
                                train_size, split_type='cblind')

# index for each set of train-test pairs
xo_train, xd_train, y_train, xo_test, xd_test, y_test = data_indexing(train_pairs,test_pairs,x_all,x_drug,y_series) # select train and test

Imports: Done
Data Imports and Wrangling: Done
cutoff percentile:  95
functional score cutoff:  0.577659576369025
Fraction of cls in sets, relative to all cls before missing values are removed
train fraction 0.7317073170731707, test fraction 0.2682926829268293
------
Fraction of cls in sets, relative to all cl drug pairs, after missing values are removed
train fraction 0.6352432571944767, test fraction 0.23286875725900116
------


### Neural Network 

In [2]:
## Model Building

# Learning rate schedule function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


## CNN using functional API
def build_model(learning_rate=1e-2, momentum=0.9, seed=42):

    # set weight initialiser
    initializer = tf.keras.initializers.GlorotUniform(seed=seed)
    
    # phosphoproteomics data input
    x_input = layers.Input(shape=(xo_train.shape[1],1))
    # 1st convolution layer
    x = layers.Conv1D(filters=8, kernel_size=4, kernel_initializer=initializer, activation='relu')(x_input) 
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.1,seed=seed)(x)
    # 2nd convolution layer
    x = layers.Conv1D(filters=16, kernel_size=4, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.1,seed=seed)(x)
    x = layers.Flatten()(x)

    # FC layer for xo_train
    x = layers.Dense(128, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5,seed=seed)(x)
    x = layers.Dense(128, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5,seed=seed)(x)
    
    # one-hot encoded drug data + dense layer
    y_input = layers.Input(shape=(xd_train.shape[1]))
    y = Dense(64, kernel_initializer=initializer, activation="relu")(y_input) 
    
    # Concatenate phosphoproteomics and encoded drug data
    z = layers.concatenate([x, y])
    z = layers.Dense(64, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(64, kernel_initializer=initializer, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dense(1, kernel_initializer=initializer)(z)

    model = tf.keras.Model([x_input, y_input], z)
    
    opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, momentum=momentum)

    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    return model

print('Model Building: Done')

Model Building: Done


In [3]:
m_func = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler
epochs = 100

train_model(m_func,lr_scheduler, 
          x_train = [xo_train, xd_train], x_test = [xo_test, xd_test], 
          y_train = y_train, y_test = y_test, 
          epochs = epochs)

Epoch 1/100
326/326 [==============================] - 19s 27ms/step - loss: 0.6894 - mae: 0.6111 - val_loss: 308.2338 - val_mae: 15.2312 - lr: 0.0100
Epoch 2/100
326/326 [==============================] - 8s 24ms/step - loss: 0.4172 - mae: 0.4954 - val_loss: 814.5728 - val_mae: 2.1061 - lr: 0.0100
Epoch 3/100
326/326 [==============================] - 8s 24ms/step - loss: 0.3776 - mae: 0.4703 - val_loss: 12307.8477 - val_mae: 11.8866 - lr: 0.0100
Epoch 4/100
326/326 [==============================] - 8s 25ms/step - loss: 0.3513 - mae: 0.4520 - val_loss: 45728.7812 - val_mae: 58.7143 - lr: 0.0100
Epoch 5/100
326/326 [==============================] - 8s 25ms/step - loss: 0.3709 - mae: 0.4649 - val_loss: 1824.0634 - val_mae: 9.1770 - lr: 0.0100
Epoch 6/100
326/326 [==============================] - 9s 27ms/step - loss: 0.3697 - mae: 0.4673 - val_loss: 1224.3070 - val_mae: 5.5992 - lr: 0.0100
Epoch 7/100
326/326 [==============================] - 8s 26ms/step - loss: 0.3533 - mae: 0.4570

In [ ]:
epochs = 100
rand_seed_list = [913,425]
model = build_model() # set model and parameters
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler) # set learning rate scheduler

train_model_cv(model,lr_scheduler,
             train_pairs,x_all,x_drug,y_series,
             epochs,rand_seed_list)